In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling
import joblib
import matplotlib.pyplot as plt
import xgboost as xgb
import json
import pickle
import category_encoders as ce
import openbabel.pybel as pybel
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils.utils import *
from utils.surface_area import *
%matplotlib inline

In [21]:
def xyz_to_smiles(fname: str) -> str:
    mol = next(pybel.readfile("xyz", fname))
    smi = mol.write(format="smi")
    return smi.split()[0].strip()

splitdot = lambda x: x.split('.')[0]

# Metal Linker

In [36]:
xyz_path = 'data/additional/metal_linker'

filenames = os.listdir(xyz_path)
filenames = list(map(splitdot, filenames))

In [37]:
smiles_metal = {}
for filename in filenames:
    mof_xyz = os.path.join(xyz_path, "{}.xyz".format(filename))
    smi = xyz_to_smiles(mof_xyz)
    
    idx = int(filename.split('_')[-1])
    smiles_metal[idx] = smi

# Organic Linker

In [38]:
xyz_path = 'data/additional/organic_linker'

filenames = os.listdir(xyz_path)
filenames = list(map(splitdot, filenames))

In [39]:
smiles_organic = {}
for filename in filenames:
    mof_xyz = os.path.join(xyz_path, "{}.xyz".format(filename))
    smi = xyz_to_smiles(mof_xyz)
    
    idx = int(filename.split('_')[-1])
    smiles_organic[idx] = smi

# Replace index with smiles

In [144]:
mode = 'test'
df = pd.read_csv('data/{}/clean_{}.csv'.format(mode, mode))
fix_linker = pd.read_csv('data/sra_2to15.csv')

mask = (fix_linker.flag == mode).values
fix_train_linker = fix_linker[mask]
fix_test_linker = fix_linker[~mask]

df = df.set_index('MOFname')
df.loc[fix_train_linker.MOFname.values, 'metal_linker'] = fix_train_linker.metal_linker.values
df.loc[fix_train_linker.MOFname.values, 'organic_linker1'] = fix_train_linker.organic_linker1.values
df.loc[fix_train_linker.MOFname.values, 'organic_linker2'] = fix_train_linker.organic_linker2.values
df = df.replace({'metal_linker': smiles_metal, 'organic_linker1': smiles_organic, 'organic_linker2': smiles_organic})
df = df.reset_index()

In [145]:
df.to_csv('data/{}/clean_{}_linker.csv'.format(mode, mode), index=False)